<a href="https://www.kaggle.com/code/enricofindley/country-similarity-calculation?scriptVersionId=141207125" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Installing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Loading

In [2]:
# Load and preprocess data
raw_data = pd.read_csv('/kaggle/input/countries-of-the-world-2023/world-data-2023.csv')
raw_data

,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
0,Afghanistan,60,AF,58.10%,"652,230","323,000",32.49,93.0,Kabul,"8,672",...,78.40%,0.28,"38,041,754",48.90%,9.30%,71.40%,11.12%,"9,797,273",33.939110,67.709953
1,Albania,105,AL,43.10%,"28,748","9,000",11.78,355.0,Tirana,"4,536",...,56.90%,1.20,"2,854,191",55.70%,18.60%,36.60%,12.33%,"1,747,593",41.153332,20.168331
2,Algeria,18,DZ,17.40%,"2,381,741","317,000",24.28,213.0,Algiers,"150,006",...,28.10%,1.72,"43,053,054",41.20%,37.20%,66.10%,11.70%,"31,510,100",28.033886,1.659626
3,Andorra,164,AD,40.00%,468,NaN,7.20,376.0,Andorra la Vella,469,...,36.40%,3.33,"77,142",NaN,NaN,NaN,NaN,"67,873",42.506285,1.521801
4,Angola,26,AO,47.50%,"1,246,700","117,000",40.73,244.0,Luanda,"34,693",...,33.40%,0.21,"31,825,295",77.50%,9.20%,49.10%,6.89%,"21,061,025",-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Venezuela,32,VE,24.50%,"912,050","343,000",17.88,58.0,Caracas,"164,175",...,45.80%,1.92,"28,515,829",59.70%,NaN,73.30%,8.80%,"25,162,368",6.423750,-66.589730
191,Vietnam,314,VN,39.30%,"331,210","522,000",16.75,84.0,Hanoi,"192,668",...,43.50%,0.82,"96,462,106",77.40%,19.10%,37.60%,2.01%,"35,332,140",14.058324,108.277199
192,Yemen,56,YE,44.60%,"527,968","40,000",30.45,967.0,Sanaa,"10,609",...,81.00%,0.31,"29,161,922",38.00%,NaN,26.60%,12.91%,"10,869,523",15.552727,48.516388
193,Zambia,25,ZM,32.10%,"752,618","16,000",36.19,260.0,Lusaka,"5,141",...,27.50%,1.19,"17,861,030",74.60%,16.20%,15.60%,11.43%,"7,871,713",-13.133897,27.849332


# EDA

In [3]:
raw_data.shape

(195, 35)

In [4]:
raw_data.describe()

,Birth Rate,Calling Code,Fertility Rate,Infant mortality,Life expectancy,Maternal mortality ratio,Physicians per thousand,Latitude,Longitude
count,189.000000,194.000000,188.000000,189.000000,187.000000,181.000000,188.000000,194.000000,194.000000
mean,20.214974,360.546392,2.698138,21.332804,72.279679,160.392265,1.839840,19.092351,20.232434
std,9.945774,323.236419,1.282267,19.548058,7.483661,233.502024,1.684261,23.961779,66.716110
min,5.900000,1.000000,0.980000,1.400000,52.800000,2.000000,0.010000,-40.900557,-175.198242
25%,11.300000,82.500000,1.705000,6.000000,67.000000,13.000000,0.332500,4.544175,-7.941496
50%,17.950000,255.500000,2.245000,14.000000,73.200000,53.000000,1.460000,17.273849,20.972652
75%,28.750000,506.750000,3.597500,32.700000,77.500000,186.000000,2.935000,40.124603,48.281523
max,46.080000,1876.000000,6.910000,84.500000,85.400000,1150.000000,8.420000,64.963051,178.065032


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Density
(P/Km2)                            195 non-null    object 
 2   Abbreviation                               188 non-null    object 
 3   Agricultural Land( %)                      188 non-null    object 
 4   Land Area(Km2)                             194 non-null    object 
 5   Armed Forces size                          171 non-null    object 
 6   Birth Rate                                 189 non-null    float64
 7   Calling Code                               194 non-null    float64
 8   Capital/Major City                         192 non-null    object 
 9   Co2-Emissions                              188 non-null    object 
 10  CPI                       

Next, we are going to select all relevant column except column <code>Country</code>,<code>Abbreviation</code>,<code>Calling Code</code>,<code>Capital/Major City</code>, <code>Currency-Code</code>, <code>Largest city</code>, <code>Official language</code>, <code>Latitude</code>,and <code>Longitude</code>

In [6]:
dropped_features = [
    'Abbreviation', 'Calling Code', 'Capital/Major City', 'Currency-Code', 'Largest city', 'Official language', 'Latitude', 'Longitude'
]
data = raw_data.drop(raw_data[dropped_features], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 27 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Density
(P/Km2)                            195 non-null    object 
 2   Agricultural Land( %)                      188 non-null    object 
 3   Land Area(Km2)                             194 non-null    object 
 4   Armed Forces size                          171 non-null    object 
 5   Birth Rate                                 189 non-null    float64
 6   Co2-Emissions                              188 non-null    object 
 7   CPI                                        178 non-null    object 
 8   CPI Change (%)                             179 non-null    object 
 9   Fertility Rate                             188 non-null    float64
 10  Forested Area (%)         

Removing <code>"%"</code>, <code>"$"</code>, and replace <code>","</code> with <code>"."</code> from data, and also convert <code>"object"</code> data type to <code>"float64"</code>

In [7]:
object_columns = data.select_dtypes(include=['object']).columns
for column in object_columns[1:]:
    data[column] = data[column].str.replace('[$,%]', '', regex=True).str.replace(',', '.').astype('float64')

In [8]:
data.head()

,Country,Density\n(P/Km2),Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,...,Maternal mortality ratio,Minimum wage,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population
0,Afghanistan,60.0,58.1,652230.0,323000.0,32.49,8672.0,149.90,2.3,4.47,...,638.0,0.43,78.4,0.28,38041754.0,48.9,9.3,71.4,11.12,9797273.0
1,Albania,105.0,43.1,28748.0,9000.0,11.78,4536.0,119.05,1.4,1.62,...,15.0,1.12,56.9,1.20,2854191.0,55.7,18.6,36.6,12.33,1747593.0
2,Algeria,18.0,17.4,2381741.0,317000.0,24.28,150006.0,151.36,2.0,3.02,...,112.0,0.95,28.1,1.72,43053054.0,41.2,37.2,66.1,11.70,31510100.0
3,Andorra,164.0,40.0,468.0,NaN,7.20,469.0,NaN,NaN,1.27,...,NaN,6.63,36.4,3.33,77142.0,NaN,NaN,NaN,NaN,67873.0
4,Angola,26.0,47.5,1246700.0,117000.0,40.73,34693.0,261.73,17.1,5.52,...,241.0,0.71,33.4,0.21,31825295.0,77.5,9.2,49.1,6.89,21061025.0


In [9]:
data.isnull().sum()

Country                                       0
Density\n(P/Km2)                              0
Agricultural Land( %)                         7
Land Area(Km2)                                1
Armed Forces size                            24
Birth Rate                                    6
Co2-Emissions                                 7
CPI                                          17
CPI Change (%)                               16
Fertility Rate                                7
Forested Area (%)                             7
Gasoline Price                               20
GDP                                           2
Gross primary education enrollment (%)        7
Gross tertiary education enrollment (%)      12
Infant mortality                              6
Life expectancy                               8
Maternal mortality ratio                     14
Minimum wage                                 45
Out of pocket health expenditure              7
Physicians per thousand                 

And doing necessary remove missing values

In [10]:
data = data.dropna()

In [11]:
data.isnull().sum()

Country                                      0
Density\n(P/Km2)                             0
Agricultural Land( %)                        0
Land Area(Km2)                               0
Armed Forces size                            0
Birth Rate                                   0
Co2-Emissions                                0
CPI                                          0
CPI Change (%)                               0
Fertility Rate                               0
Forested Area (%)                            0
Gasoline Price                               0
GDP                                          0
Gross primary education enrollment (%)       0
Gross tertiary education enrollment (%)      0
Infant mortality                             0
Life expectancy                              0
Maternal mortality ratio                     0
Minimum wage                                 0
Out of pocket health expenditure             0
Physicians per thousand                      0
Population   

print the correlation matrix

In [12]:
data.corr()

/tmp/ipykernel_20/2627137660.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,Density\n(P/Km2),Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,Forested Area (%),...,Maternal mortality ratio,Minimum wage,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population
Density\n(P/Km2),1.000000,0.103444,-0.168403,0.090658,-0.225131,0.010688,-0.076595,-0.112570,-0.231984,-0.202357,...,-0.120651,0.136731,0.152197,0.037323,0.126830,-0.122617,0.122078,-0.086935,-0.183332,0.073141
Agricultural Land( %),0.103444,1.000000,-0.094716,0.048109,0.046887,0.049251,0.032455,0.051799,0.036202,-0.479528,...,0.084137,-0.029806,0.097855,0.051420,0.114330,-0.118852,0.031236,0.017467,0.067184,0.090362
Land Area(Km2),-0.168403,-0.094716,1.000000,0.578881,-0.115833,0.584613,0.052800,0.070576,-0.100768,0.043783,...,-0.095196,0.125973,-0.075367,0.107889,0.428854,0.033591,-0.169455,0.155892,0.040641,0.532928
Armed Forces size,0.090658,0.048109,0.578881,1.000000,-0.168143,0.780649,0.032275,0.065756,-0.167841,-0.015342,...,-0.114708,-0.035772,0.137614,-0.006254,0.919811,-0.118941,-0.203833,0.160193,-0.005889,0.892436
Birth Rate,-0.225131,0.046887,-0.115833,-0.168143,1.000000,-0.182976,0.219490,0.181648,0.981658,-0.029042,...,0.779920,-0.506853,0.234615,-0.742452,-0.079044,0.247753,-0.414078,0.133590,-0.038755,-0.136104
Co2-Emissions,0.010688,0.049251,0.584613,0.780649,-0.182976,1.000000,-0.029472,0.003278,-0.161215,-0.008080,...,-0.123734,0.075313,-0.057482,0.061681,0.807754,0.004417,-0.158487,0.119886,0.004604,0.925983
CPI,-0.076595,0.032455,0.052800,0.032275,0.219490,-0.029472,1.000000,0.671789,0.206345,-0.129067,...,0.171651,-0.186609,0.164236,-0.189877,0.018650,-0.092104,-0.208027,0.075390,0.207450,0.000960
CPI Change (%),-0.112570,0.051799,0.070576,0.065756,0.181648,0.003278,0.671789,1.000000,0.158901,-0.148543,...,0.203222,-0.144479,0.109790,-0.116968,0.032283,-0.111755,-0.252480,0.330572,0.180209,0.028532
Fertility Rate,-0.231984,0.036202,-0.100768,-0.167841,0.981658,-0.161215,0.206345,0.158901,1.000000,-0.029249,...,0.789770,-0.442311,0.197496,-0.677120,-0.072807,0.238205,-0.415995,0.142781,-0.060085,-0.121728
Forested Area (%),-0.202357,-0.479528,0.043783,-0.015342,-0.029042,-0.008080,-0.129067,-0.148543,-0.029249,1.000000,...,0.043882,-0.017925,-0.241541,-0.096430,-0.034755,0.279226,-0.051366,0.017522,-0.147384,-0.008241


# Similarity Calculation

In [13]:
target_country = 'United States'
# Select the target country's data (replace 'Target Country' with the actual country name)
data_search = data[data['Country'] != target_country]
target_country_data = data[data['Country'] == target_country].iloc[0, 1:].values

# Calculate cosine similarity with other countries
similarities = cosine_similarity(target_country_data.reshape(1, -1), data_search.iloc[:, 1:].values)

# Find the most similar countries
num_similar_countries = 5  # You can adjust the number of similar countries
similar_country_indices = np.argsort(similarities[0])[-num_similar_countries:][::-1]

print(num_similar_countries,"Countries That Most Similar To", target_country, "is:")
for index in similar_country_indices:
    country_name = data.iloc[index, 0]  # Assuming the country names are in the first column
    similarity_score = similarities[0][index]  # Fetch the similarity score
    print(f"Country: {country_name}, Similarity Score: {similarity_score}")

5 Countries That Most Similar To United States is:
Country: Republic of Ireland, Similarity Score: 0.9999999999865355
Country: Netherlands, Similarity Score: 0.999999999980952
Country: Australia, Similarity Score: 0.9999999999772671
Country: Germany, Similarity Score: 0.9999999999717075
Country: Luxembourg, Similarity Score: 0.9999999999694797
